In [1]:
from transformers import AutoTokenizer
from model import NERModel
from dataset import CoNLLDataModule
import pytorch_lightning as pl
import json
from argparse import Namespace

/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
config = Namespace()
vars(config).update(json.load(open("checkpoints/config.json","r")))
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = NERModel.load_from_checkpoint("checkpoints/epoch=9-step=11709.ckpt", config=config, tokenizer=tokenizer, test=True)

In [3]:
model("I moved to Mahattan last year with my sister, Sovia")

[{'word': 'mahattan', 'score': 0.99996495, 'label': 'LOCATION'},
 {'word': 'sovia', 'score': 0.9997329, 'label': 'PERSON'}]

In [4]:
trainer = pl.Trainer()
ner_dm = CoNLLDataModule(config, tokenizer)
trainer.test(model, datamodule=ner_dm)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:596: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Reusing dataset conll2003 (/home/phuctth/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)
/home/phuctth/miniconda3/envs/limit-bert/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


val/PER-F1: 0.967, val/ORG-F1: 0.887, val/LOC-F1: 0.925, val/MISC-F1: 0.795, 
F1: 0.9096040215186524

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]